In [ ]:
import cv2
import dlib
import numpy as np
import os
import pandas as pd # Import pandas for CSV handling

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat"
)

# Define the root directories for input and output
root_input_dir = "archive_3"
root_output_dir = "processed_faces_newFinal8"

# Define the scaling factor for upscaling images
scale_factor = 12

# Create the root output directory if it doesn't exist
os.makedirs(root_output_dir, exist_ok=True)

# List of emotion folders
emotion_folders = ["anger", "contempt", "disgust", "happy", "sadness", "fear", "surprise"]

# List to store all landmark and fitted shape data for CSV export
all_facial_features_data = [] # Renamed for clarity to include more than just raw landmarks

print("Starting facial landmark detection, shape drawing, and feature extraction...")
print("Output images will be split into '_with_photo', '_without_photo', and '_shapes_drawn' folders.")
print("A comprehensive CSV file with landmark coordinates AND fitted shape parameters will be generated.")

# --- Helper Function for Drawing Shapes (from previous correction) ---
def draw_facial_shapes(image, landmarks):
    """
    Draws different geometric shapes on a given image based on facial landmarks.
    Args:
        image (numpy.ndarray): The image on which to draw.
        landmarks (dlib.full_object_detection): Dlib's landmark object.
    Returns:
        numpy.ndarray: The image with shapes drawn.
    """
    drawn_image = image.copy()

    # Define colors for different shapes (BGR format)
    color_eyebrows = (0, 255, 0)   # Green
    color_eyes = (255, 0, 0)       # Blue
    color_nose_bridge = (0, 255, 255) # Yellow
    color_nose_tip = (255, 255, 0) # Cyan
    color_lips = (0, 165, 255)     # Orange
    color_cheeks = (128, 0, 128)   # Purple
    thickness = 2 # Line thickness

    # --- Eyebrows (Approximated by polylines) ---
    # Left Eyebrow: 17-21
    for i in range(17, 21):
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # Right Eyebrow: 22-26
    for i in range(22, 26):
        pt1 = (landmarks.part(i).x, landmarks.part(i).y)
        pt2 = (landmarks.part(i + 1).x, landmarks.part(i + 1).y)
        cv2.line(drawn_image, pt1, pt2, color_eyebrows, thickness)

    # --- Eyes (Ellipses) ---
    # Left Eye: 36-41
    left_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)])
    if len(left_eye_points) >= 5: # cv2.fitEllipse requires at least 5 points
        try: # Add try-except for robustness against poor point configurations
            (center_le, axes_le, angle_le) = cv2.fitEllipse(left_eye_points)
            if axes_le[0] > 0 and axes_le[1] > 0: # Ensure axes are valid
                cv2.ellipse(drawn_image, (int(center_le[0]), int(center_le[1])),
                            (int(axes_le[0] / 2), int(axes_le[1] / 2)),
                            angle_le, 0, 360, color_eyes, thickness)
        except cv2.error:
            # print(f"Warning: Could not fit ellipse for left eye. Skipping drawing.")
            pass # Suppress drawing error, fitting will use NaN

    # Right Eye: 42-47
    right_eye_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)])
    if len(right_eye_points) >= 5:
        try:
            (center_re, axes_re, angle_re) = cv2.fitEllipse(right_eye_points)
            if axes_re[0] > 0 and axes_re[1] > 0:
                cv2.ellipse(drawn_image, (int(center_re[0]), int(center_re[1])),
                            (int(axes_re[0] / 2), int(axes_re[1] / 2)),
                            angle_re, 0, 360, color_eyes, thickness)
        except cv2.error:
            # print(f"Warning: Could not fit ellipse for right eye. Skipping drawing.")
            pass # Suppress drawing error, fitting will use NaN


    # --- Nose ---
    # Nose Bridge (Trapezium): Using landmarks 27, 28, 29, 30
    nose_bridge_pts = np.array([
        (landmarks.part(27).x, landmarks.part(27).y),
        (landmarks.part(28).x, landmarks.part(28).y),
        (landmarks.part(29).x, landmarks.part(29).y),
        (landmarks.part(30).x, landmarks.part(30).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_bridge_pts], True, color_nose_bridge, thickness)

    # Nose Tip (Triangle): Using landmarks 30, 31, 35
    nose_tip_pts = np.array([
        (landmarks.part(30).x, landmarks.part(30).y),
        (landmarks.part(31).x, landmarks.part(31).y),
        (landmarks.part(35).x, landmarks.part(35).y)
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [nose_tip_pts], True, color_nose_tip, thickness)

    # --- Lips (Outer Lip Ellipse) ---
    # Outer Lip: 48-59
    outer_lip_points = np.array([(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)])
    if len(outer_lip_points) >= 5:
        try:
            (center_ol, axes_ol, angle_ol) = cv2.fitEllipse(outer_lip_points)
            if axes_ol[0] > 0 and axes_ol[1] > 0:
                cv2.ellipse(drawn_image, (int(center_ol[0]), int(center_ol[1])),
                            (int(axes_ol[0] / 2), int(axes_ol[1] / 2)),
                            angle_ol, 0, 360, color_lips, thickness)
        except cv2.error:
            # print(f"Warning: Could not fit ellipse for outer lip. Skipping drawing.")
            pass # Suppress drawing error, fitting will use NaN

    # --- Cheeks (Approximated by Broader Triangles) ---
    # Left Cheek: Using outer eye corner (36), outer jawline (2), left mouth corner (48)
    left_cheek_pts = np.array([
        (landmarks.part(36).x, landmarks.part(36).y), # Left outer eye corner
        (landmarks.part(2).x, landmarks.part(2).y),   # Left jawline (near chin)
        (landmarks.part(48).x, landmarks.part(48).y)  # Left mouth corner
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [left_cheek_pts], True, color_cheeks, thickness)

    # Right Cheek: Using outer eye corner (45), outer jawline (14), right mouth corner (54)
    right_cheek_pts = np.array([
        (landmarks.part(45).x, landmarks.part(45).y), # Right outer eye corner
        (landmarks.part(14).x, landmarks.part(14).y), # Right jawline (near chin)
        (landmarks.part(54).x, landmarks.part(54).y)  # Right mouth corner
    ], np.int32).reshape((-1, 1, 2))
    cv2.polylines(drawn_image, [right_cheek_pts], True, color_cheeks, thickness)

    return drawn_image

# --- Helper Functions for Curve Fitting ---

def fit_parabola_params(points):
    """
    Fits a 2nd-degree polynomial (parabola) to a set of (x, y) points.
    Returns (a, b, c) for y = ax^2 + bx + c.
    Returns (NaN, NaN, NaN) if fitting fails.
    """
    if len(points) < 3: # Need at least 3 points to fit a parabola
        return np.nan, np.nan, np.nan
    x_coords = np.array([p[0] for p in points])
    y_coords = np.array([p[1] for p in points])
    try:
        # polyfit can sometimes raise warnings for ill-conditioned matrices
        # Suppress warnings if you are sure about the input data
        with np.errstate(all='raise'):
            coefficients = np.polyfit(x_coords, y_coords, 2)
            return tuple(coefficients) # (a, b, c)
    except Exception as e: # Catch numpy polynomial fitting errors
        # print(f"Warning: Parabola fitting failed: {e}")
        return np.nan, np.nan, np.nan

def fit_ellipse_params(points):
    """
    Fits an ellipse to a set of (x, y) points using cv2.fitEllipse.
    Returns (center_x, center_y, major_axis, minor_axis, angle)
    Returns (NaN, NaN, NaN, NaN, NaN) if fitting fails or points are insufficient.
    """
    if len(points) < 5: # cv2.fitEllipse requires at least 5 points
        return np.nan, np.nan, np.nan, np.nan, np.nan
    points_np = np.array(points, dtype=np.int32)
    try:
        (center, axes, angle) = cv2.fitEllipse(points_np)
        # Ensure axes are positive; sometimes very small or negative can be returned
        if axes[0] <= 0 or axes[1] <= 0:
            return np.nan, np.nan, np.nan, np.nan, np.nan

        # axes are (major_axis_length, minor_axis_length) or vice-versa.
        # It's diameter, so convert to radius if needed for other applications.
        # For now, we'll store diameter values as returned.
        return center[0], center[1], axes[0], axes[1], angle
    except cv2.error as e:
        # print(f"Warning: Ellipse fitting failed: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan
    except Exception as e: # Catch any other unexpected errors
        # print(f"Warning: Ellipse fitting failed unexpectedly: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan


# --- Main Processing Loop ---
for emotion in emotion_folders:
    input_emotion_path = os.path.join(root_input_dir, emotion)

    # Define the new output subfolders for each emotion
    output_with_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_with_photo")
    output_without_photo_path = os.path.join(root_output_dir, emotion, f"{emotion}_without_photo")
    output_shapes_drawn_path = os.path.join(root_output_dir, emotion, f"{emotion}_shapes_drawn")

    # Create these new output subdirectories if they don't exist
    os.makedirs(output_with_photo_path, exist_ok=True)
    os.makedirs(output_without_photo_path, exist_ok=True)
    os.makedirs(output_shapes_drawn_path, exist_ok=True)

    print(f"\nProcessing emotion: {emotion}...")

    for filename in os.listdir(input_emotion_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(input_emotion_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping.")
                continue

            print(f"  Processing file: {filename}")

            upscaled_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

            faces = detector(gray)

            if len(faces) > 0:
                face = faces[0]
                landmarks = predictor(gray, face)

                # --- Initialize row for CSV data ---
                current_image_features = {'emotion': emotion, 'filename': filename}

                # --- Collect raw landmark data for CSV ---
                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y
                    current_image_features[f'landmark_{n}_x'] = x
                    current_image_features[f'landmark_{n}_y'] = y

                # --- Apply Curve Fitting Algorithms and Store Parameters ---

                # Left Eyebrow (Parabola: y = ax^2 + bx + c)
                leb_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(17, 22)]
                a_leb, b_leb, c_leb = fit_parabola_params(leb_points)
                current_image_features['leb_parabola_a'] = a_leb
                current_image_features['leb_parabola_b'] = b_leb
                current_image_features['leb_parabola_c'] = c_leb

                # Right Eyebrow (Parabola: y = ax^2 + bx + c)
                reb_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(22, 27)]
                a_reb, b_reb, c_reb = fit_parabola_params(reb_points)
                current_image_features['reb_parabola_a'] = a_reb
                current_image_features['reb_parabola_b'] = b_reb
                current_image_features['reb_parabola_c'] = c_reb

                # Left Eye (Ellipse: center_x, center_y, major_axis, minor_axis, angle)
                le_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(36, 42)]
                cx_le, cy_le, maj_le, min_le, ang_le = fit_ellipse_params(le_points)
                current_image_features['le_ellipse_center_x'] = cx_le
                current_image_features['le_ellipse_center_y'] = cy_le
                current_image_features['le_ellipse_major_axis'] = maj_le
                current_image_features['le_ellipse_minor_axis'] = min_le
                current_image_features['le_ellipse_angle'] = ang_le

                # Right Eye (Ellipse)
                re_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(42, 48)]
                cx_re, cy_re, maj_re, min_re, ang_re = fit_ellipse_params(re_points)
                current_image_features['re_ellipse_center_x'] = cx_re
                current_image_features['re_ellipse_center_y'] = cy_re
                current_image_features['re_ellipse_major_axis'] = maj_re
                current_image_features['re_ellipse_minor_axis'] = min_re
                current_image_features['re_ellipse_angle'] = ang_re

                # Outer Lip (Ellipse)
                ol_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 60)]
                cx_ol, cy_ol, maj_ol, min_ol, ang_ol = fit_ellipse_params(ol_points)
                current_image_features['ol_ellipse_center_x'] = cx_ol
                current_image_features['ol_ellipse_center_y'] = cy_ol
                current_image_features['ol_ellipse_major_axis'] = maj_ol
                current_image_features['ol_ellipse_minor_axis'] = min_ol
                current_image_features['ol_ellipse_angle'] = ang_ol

                # Nose Bridge (Trapezium - extract vertices)
                current_image_features['nose_bridge_v1_x'] = landmarks.part(27).x
                current_image_features['nose_bridge_v1_y'] = landmarks.part(27).y
                current_image_features['nose_bridge_v2_x'] = landmarks.part(28).x
                current_image_features['nose_bridge_v2_y'] = landmarks.part(28).y
                current_image_features['nose_bridge_v3_x'] = landmarks.part(29).x
                current_image_features['nose_bridge_v3_y'] = landmarks.part(29).y
                current_image_features['nose_bridge_v4_x'] = landmarks.part(30).x
                current_image_features['nose_bridge_v4_y'] = landmarks.part(30).y

                # Nose Tip (Triangle - extract vertices)
                current_image_features['nose_tip_v1_x'] = landmarks.part(30).x
                current_image_features['nose_tip_v1_y'] = landmarks.part(30).y
                current_image_features['nose_tip_v2_x'] = landmarks.part(31).x
                current_image_features['nose_tip_v2_y'] = landmarks.part(31).y
                current_image_features['nose_tip_v3_x'] = landmarks.part(35).x
                current_image_features['nose_tip_v3_y'] = landmarks.part(35).y

                # Left Cheek (Triangle - extract vertices)
                current_image_features['left_cheek_v1_x'] = landmarks.part(36).x
                current_image_features['left_cheek_v1_y'] = landmarks.part(36).y
                current_image_features['left_cheek_v2_x'] = landmarks.part(2).x
                current_image_features['left_cheek_v2_y'] = landmarks.part(2).y
                current_image_features['left_cheek_v3_x'] = landmarks.part(48).x
                current_image_features['left_cheek_v3_y'] = landmarks.part(48).y

                # Right Cheek (Triangle - extract vertices)
                current_image_features['right_cheek_v1_x'] = landmarks.part(45).x
                current_image_features['right_cheek_v1_y'] = landmarks.part(45).y
                current_image_features['right_cheek_v2_x'] = landmarks.part(14).x
                current_image_features['right_cheek_v2_y'] = landmarks.part(14).y
                current_image_features['right_cheek_v3_x'] = landmarks.part(54).x
                current_image_features['right_cheek_v3_y'] = landmarks.part(54).y
                
                
                

                # Add the collected and fitted data row to the main list
                all_facial_features_data.append(current_image_features)

                # --- Prepare images for all output types (visualizations) ---
                drawn_image_with_photo = upscaled_image.copy()
                drawn_image_without_photo = np.zeros(upscaled_image.shape, dtype=np.uint8)
                drawn_image_shapes_only = np.zeros(upscaled_image.shape, dtype=np.uint8)

                # Get position of landmark 31 for quadrant lines (on _with_photo image)
                nose_x = landmarks.part(30).x
                nose_y = landmarks.part(30).y
                img_h, img_w = drawn_image_with_photo.shape[:2]
                cv2.line(drawn_image_with_photo, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1)
                cv2.line(drawn_image_with_photo, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1)

                drawn_index = 1
                seen = set()

                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y

                    if (x, y) in seen:
                        continue
                    seen.add((x, y))

                    circle_radius = 2
                    fill_type = -1

                    # Offset for label (re-using your existing logic)
                    offset_x, offset_y = -6, 10
                    if n in [37, 38, 39, 43, 44, 45]: # Upper eye points
                        offset_y = -10
                    elif n == 49: # Left mouth corner
                        offset_x = -20
                        offset_y = 6
                    elif n == 50: # Top lip center
                        offset_x = 0
                        offset_y = -8
                    elif n == 65: # Chin left
                        offset_x = -14
                        offset_y = 10
                    elif 48 <= n <= 54: # Upper lip
                        offset_y = -8
                    elif 55 <= n <= 59: # Lower lip center to right
                        offset_y = 10
                    elif 60 <= n <= 64: # Inner mouth top
                        offset_y = -8
                    elif 65 <= n <= 67: # Chin right
                        offset_y = 10

                    # Draw on the image WITH photo (original background)
                    text_color_with_photo = (0, 0, 0) # Black text
                    cv2.circle(drawn_image_with_photo, (x, y), circle_radius, (0, 0, 255), fill_type) # Red circles
                    cv2.putText(drawn_image_with_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, text_color_with_photo, 1, cv2.LINE_AA)

                    # Draw on the image WITHOUT photo (black background)
                    text_color_without_photo = (255, 255, 255) # White text/circles
                    cv2.circle(drawn_image_without_photo, (x, y), circle_radius, text_color_without_photo, fill_type)
                    cv2.putText(drawn_image_without_photo, str(drawn_index), (x + offset_x, y + offset_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.35, text_color_without_photo, 1, cv2.LINE_AA)

                    drawn_index += 1

                # --- Draw the specific shapes on the dedicated 'shapes_only' image ---
                drawn_image_shapes_only = draw_facial_shapes(drawn_image_shapes_only, landmarks)

                # Save the processed images to their respective folders
                output_image_path_with_photo = os.path.join(output_with_photo_path, filename)
                cv2.imwrite(output_image_path_with_photo, drawn_image_with_photo)

                output_image_path_without_photo = os.path.join(output_without_photo_path, filename)
                cv2.imwrite(output_image_path_without_photo, drawn_image_without_photo)

                output_image_path_shapes_drawn = os.path.join(output_shapes_drawn_path, filename)
                cv2.imwrite(output_image_path_shapes_drawn, drawn_image_shapes_only)

            else:
                print(f"  No face detected in {filename} from {emotion} folder.")

print("\nAll emotion folders and photos processed successfully!")

# --- Save all collected facial feature data to a single CSV file ---
if all_facial_features_data:
    df = pd.DataFrame(all_facial_features_data)
    # Ensure all columns are present, even if some fits failed for certain images (filled with NaN)
    # CORRECTED LINE: Removed 'axis=1'
    df = df.reindex(columns=sorted(df.columns)) # Optional: sort columns alphabetically
    csv_output_path = os.path.join(root_output_dir, "facial_features_and_landmarks_data.csv")
    df.to_csv(csv_output_path, index=False)
    print(f"All facial landmark coordinates and fitted shape parameters saved to: {csv_output_path}")
else:
    print("No facial feature data was collected (no faces detected or no images processed).")

Starting facial landmark detection, shape drawing, and feature extraction...
Output images will be split into '_with_photo', '_without_photo', and '_shapes_drawn' folders.
A comprehensive CSV file with landmark coordinates AND fitted shape parameters will be generated.

Processing emotion: anger...
  Processing file: S010_004_00000017.png
  Processing file: S010_004_00000018.png
  Processing file: S010_004_00000019.png
  Processing file: S011_004_00000019.png
  Processing file: S011_004_00000020.png
  Processing file: S011_004_00000021.png
  Processing file: S014_003_00000028.png
  Processing file: S014_003_00000029.png
  Processing file: S014_003_00000030.png
  Processing file: S022_005_00000030.png
  Processing file: S022_005_00000031.png
  Processing file: S022_005_00000032.png
  Processing file: S026_003_00000013.png
  Processing file: S026_003_00000014.png
  Processing file: S026_003_00000015.png
  Processing file: S028_001_00000022.png
  Processing file: S028_001_00000023.png
  P